In [ ]:
import pennylane as qml
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def is_coprime(a, N):  
    return np.gcd(a,N) == 1

def is_odd(r):
    return r%2 != 0
    
def is_not_one(x, N):
    return x % N != 1 and x*x % N != 1


def shor(N):
    while True:
        value = np.random.randint(2,N-2)

        if is_coprime(value, N):
            return [value, N/value]
            
        U = get_matrix_a_mod_N(value, N)
        period = get_period(U, value)
        
        if is_odd(period):
            continue
        
        x = value**(period/2)%N
        
        if is_not_one(x, N):
            p = np.gcd(x-1, N)
            q = np.gcd(x+1, N)
            return [p,q]

print(shor(21))
